# Generate Data

In [1]:
# Set Up
import pandas as pd
import numpy as np
from datetime import timedelta
import random

np.random.seed(42)
random.seed(42)

In [2]:
# Dimension Tables
## Branch
dim_branch = pd.DataFrame({
    "branch_id": [f"B{i:02d}" for i in range(1, 16)],
    "branch_name": [f"SME Center {i}" for i in range(1, 16)],
    "region": np.random.choice(["Barat", "Tengah", "Timur"], 15, p=[0.35, 0.45, 0.20]),
    "city": np.random.choice(
        ["Jakarta", "Bandung", "Semarang", "Surabaya", "Makassar", "Medan"], 15
    ),
    "branch_type": ["SME Center"] * 15
})
## RM
dim_rm = pd.DataFrame({
    "rm_id": [f"RM{i:03d}" for i in range(1, 31)],
    "rm_level": np.random.choice(["Junior", "Senior"], 30, p=[0.55, 0.45]),
    "rm_join_year": np.random.randint(2012, 2023, 30),
    "rm_status": "Active"
})
## Business Segment
dim_business_segment = pd.DataFrame({
    "business_sector": [
        "Perdagangan", "Jasa", "Produksi",
        "Pertanian", "Konstruksi", "Transportasi"
    ],
    "risk_level": [
        "Medium", "Low", "Medium",
        "High", "High", "Medium"
    ],
    "typical_plafond_range": [
        "4-7M", "1-4M", "7-10M",
        "1-4M", "7-10M", "4-7M"
    ]
})

In [12]:
# Cek dim_branch
print(dim_branch.head())
print(" ")
print(dim_branch.info())
print(" ")
print(dim_branch.nunique())
print(" ")
print(dim_branch.duplicated().sum())

  branch_id   branch_name  region      city branch_type
0       B01  SME Center 1  Tengah  Makassar  SME Center
1       B02  SME Center 2   Timur   Jakarta  SME Center
2       B03  SME Center 3  Tengah  Surabaya  SME Center
3       B04  SME Center 4  Tengah   Bandung  SME Center
4       B05  SME Center 5   Barat     Medan  SME Center
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   branch_id    15 non-null     object
 1   branch_name  15 non-null     object
 2   region       15 non-null     object
 3   city         15 non-null     object
 4   branch_type  15 non-null     object
dtypes: object(5)
memory usage: 732.0+ bytes
None
 
branch_id      15
branch_name    15
region          3
city            6
branch_type     1
dtype: int64
 
0


In [11]:
# Cek dim_rm
print(dim_rm.head())
print(" ")
print(dim_rm.info())
print(" ")
print(dim_rm.nunique())
print(" ")
print(dim_rm.duplicated().sum())

   rm_id rm_level  rm_join_year rm_status
0  RM001   Junior          2019    Active
1  RM002   Senior          2018    Active
2  RM003   Junior          2020    Active
3  RM004   Senior          2019    Active
4  RM005   Junior          2016    Active
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rm_id         30 non-null     object
 1   rm_level      30 non-null     object
 2   rm_join_year  30 non-null     int64 
 3   rm_status     30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.1+ KB
None
 
rm_id           30
rm_level         2
rm_join_year     9
rm_status        1
dtype: int64
 
0


In [10]:
# Cek dim_business_segment
print(dim_business_segment.head())
print(" ")
print(dim_business_segment.info())
print(" ")
print(dim_business_segment.nunique())
print(" ")
print(dim_business_segment.duplicated().sum())

  business_sector risk_level typical_plafond_range
0     Perdagangan     Medium                  4-7M
1            Jasa        Low                  1-4M
2        Produksi     Medium                 7-10M
3       Pertanian       High                  1-4M
4      Konstruksi       High                 7-10M
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   business_sector        6 non-null      object
 1   risk_level             6 non-null      object
 2   typical_plafond_range  6 non-null      object
dtypes: object(3)
memory usage: 276.0+ bytes
None
 
business_sector          6
risk_level               3
typical_plafond_range    3
dtype: int64
 
0


In [13]:
# Fact Credit Application
## Generate Base Structure
n = 1200
start_date = pd.to_datetime("2024-01-01")

fact = pd.DataFrame({
    "application_id": [f"APP{i:05d}" for i in range(1, n + 1)],
    "application_date": start_date + pd.to_timedelta(
        np.random.randint(0, 360, n), unit="D"
    ),
    "rm_id": np.random.choice(dim_rm["rm_id"], n),
    "branch_id": np.random.choice(dim_branch["branch_id"], n),
    "business_sector": np.random.choice(
        dim_business_segment["business_sector"],
        n,
        p=[0.35, 0.25, 0.15, 0.1, 0.1, 0.05]
    )
})
## Status Pipeline
fact["final_credit_status"] = np.random.choice(
    ["Approved", "Pending", "Rejected"],
    n,
    p=[0.55, 0.25, 0.20]
)

fact["approval_stage"] = np.where(
    fact["final_credit_status"] == "Approved",
    np.random.choice(["Analisis", "Komite"], n, p=[0.4, 0.6]),
    np.random.choice(["Verifikasi", "Analisis"], n, p=[0.6, 0.4])
)
## Dates (LOGIS BANK)
fact["decision_date"] = fact.apply(
    lambda r: r["application_date"] + timedelta(days=random.randint(5, 40))
    if r["final_credit_status"] != "Pending" else pd.NaT,
    axis=1
)

fact["akad_signing_date"] = fact.apply(
    lambda r: r["decision_date"] + timedelta(days=random.randint(3, 15))
    if r["final_credit_status"] == "Approved" else pd.NaT,
    axis=1
)

fact["disbursement_date"] = fact.apply(
    lambda r: r["akad_signing_date"] + timedelta(days=random.randint(1, 10))
    if r["final_credit_status"] == "Approved" and random.random() > 0.15 else pd.NaT,
    axis=1
)
## Plafond (LOG-NORMAL + OUTLIER)
fact["requested_plafond"] = (
    np.random.lognormal(mean=14.5, sigma=0.6, size=n) / 1_000_000
).round(2)

fact["approved_plafond"] = np.where(
    fact["final_credit_status"] == "Approved",
    (fact["requested_plafond"] * np.random.uniform(0.6, 1.0, n)).round(2),
    0
)
## Data Quality (REALISTIS)
fact["missing_field_count"] = np.random.choice(
    [0, 1, 2, 3, 4, 5],
    n,
    p=[0.35, 0.25, 0.18, 0.12, 0.07, 0.03]
)

fact["doc_completeness_flag"] = np.where(
    fact["missing_field_count"] <= 1, "Complete", "Incomplete"
)

fact["data_quality_score"] = (
    100
    - (fact["missing_field_count"] * 15)
    - np.where(fact["final_credit_status"] == "Rejected", 10, 0)
    + np.where(fact["final_credit_status"] == "Approved", 5, 0)
)

fact["data_quality_score"] = fact["data_quality_score"].clip(0, 100)
## Rejection Reason (OPTIONAL)
fact["rejection_reason"] = np.where(
    fact["final_credit_status"] == "Rejected",
    np.random.choice(
        ["Dokumen Tidak Lengkap", "Cashflow Lemah", "Sektor Risiko Tinggi", "Agunan Tidak Cukup"],
        n
    ),
    None
)

In [15]:
# Cek info
print(fact.info())
print(" ")
print(fact['application_id'].is_unique)
print(" ")
print(fact.isnull().sum())
print(" ")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   application_id         1200 non-null   object        
 1   application_date       1200 non-null   datetime64[ns]
 2   rm_id                  1200 non-null   object        
 3   branch_id              1200 non-null   object        
 4   business_sector        1200 non-null   object        
 5   final_credit_status    1200 non-null   object        
 6   approval_stage         1200 non-null   object        
 7   decision_date          893 non-null    datetime64[ns]
 8   akad_signing_date      668 non-null    datetime64[ns]
 9   disbursement_date      580 non-null    datetime64[ns]
 10  requested_plafond      1200 non-null   float64       
 11  approved_plafond       1200 non-null   float64       
 12  missing_field_count    1200 non-null   int64         
 13  doc

In [16]:
## Validasi
set(fact['rm_id']) - set(dim_rm['rm_id'])
set(fact['branch_id']) - set(dim_branch['branch_id'])
set(fact['business_sector']) - set(dim_business_segment['business_sector'])

set()

In [17]:
# Pending tapi punya decision
fact[(fact['final_credit_status']=='Pending') & fact['decision_date'].notna()]

# Rejected tapi punya akad
fact[(fact['final_credit_status']=='Rejected') & fact['akad_signing_date'].notna()]

# Tanggal tidak logis
fact[fact['decision_date'] < fact['application_date']]

,application_id,application_date,rm_id,branch_id,business_sector,final_credit_status,approval_stage,decision_date,akad_signing_date,disbursement_date,requested_plafond,approved_plafond,missing_field_count,doc_completeness_flag,data_quality_score,rejection_reason


In [18]:
fact[['requested_plafond', 'approved_plafond']].describe()

,requested_plafond,approved_plafond
count,1200.000000,1200.000000
mean,2.382267,1.044417
std,1.570470,1.330289
min,0.320000,0.000000
25%,1.330000,0.000000
50%,1.990000,0.690000
75%,2.982500,1.650000
max,17.190000,11.480000


In [19]:
fact[
    (fact['approved_plafond'] > fact['requested_plafond']) |
    ((fact['final_credit_status'] != 'Approved') & (fact['approved_plafond'] > 0))
]

,application_id,application_date,rm_id,branch_id,business_sector,final_credit_status,approval_stage,decision_date,akad_signing_date,disbursement_date,requested_plafond,approved_plafond,missing_field_count,doc_completeness_flag,data_quality_score,rejection_reason


In [20]:
fact['requested_plafond'].quantile([0.95, 0.99])

0.95    5.2220
0.99    8.2862
Name: requested_plafond, dtype: float64

In [24]:
fact['final_credit_status'].value_counts(normalize=True)

final_credit_status
Approved    0.556667
Pending     0.255833
Rejected    0.187500
Name: proportion, dtype: float64

In [23]:
fact['data_quality_score'].between(0,100).all()

np.True_

In [25]:
# OUTLIER DISENGAJA
# Pending sangat lama
outlier_idx = fact.sample(10).index
fact.loc[outlier_idx, "decision_date"] = (
    fact.loc[outlier_idx, "application_date"] + pd.to_timedelta(90, unit="D")
)

# Data quality tinggi tapi reject
idx = fact[fact["final_credit_status"] == "Rejected"].sample(8).index
fact.loc[idx, "data_quality_score"] = np.random.randint(75, 90, len(idx))

In [26]:
# Save and Download
fact.to_csv("fact_credit_application.csv", index=False)
dim_rm.to_csv("dim_rm.csv", index=False)
dim_branch.to_csv("dim_branch.csv", index=False)
dim_business_segment.to_csv("dim_business_segment.csv", index=False)

print("✅ Dummy data berhasil dibuat (4 CSV files)")

✅ Dummy data berhasil dibuat (4 CSV files)
